### Import module, function and data

In [16]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from pprint import pprint
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier # rf분류기

In [17]:
#### training_validation 나누기!!!
def separating(X,y):
    np.random.seed(7)
    index_where=[]
    index_choosed=[]
    index_boolean=np.zeros(len(y),np.bool)
    for i in range(4):
        index_where.append(np.where(y==i)[0])
    for j in range(4):
        index_choosed.append(np.random.choice(index_where[j],int(sum(y==j)*0.9),replace = False))
    for k in range(4):
        print(index_choosed[k])
        index_boolean[index_choosed[k]]=1
    return X[index_boolean], X[index_boolean==0],y[index_boolean], y[index_boolean==0]

In [18]:
def F1(y_pred, dtrain):
    labels = dtrain.get_label()
    
    pre = precision_score(y_true = labels, y_pred = y_pred, average=None)
    rec = recall_score(y_true = labels, y_pred = y_pred, average=None)
    f1_score = 8/(sum(1/pre) + sum(1/rec))

    return 'f1', f1_score

In [55]:
test_data = pd.read_csv('X_test_final.csv')
X_test = pd.concat((pd.read_csv('test_party_differ.csv').iloc[:,1],test_data),axis = 1)

In [19]:
train_data=pd.read_csv('X_train_final.csv')
test_data = pd.read_csv('X_test_final.csv')

#마지막에 추가된 변수를 붙여줌
X_train = pd.concat((pd.read_csv('train_party_differ.csv').iloc[:,1],train_data),axis = 1)
X_test = pd.concat((pd.read_csv('test_party_differ.csv').iloc[:,1],test_data),axis = 1)

In [20]:
X_train.to_csv('X_train_final.csv',index = False)
X_test.to_csv('X_test_final.csv',index = False)

In [5]:
#### load class
train_label = pd.read_csv('temp_data/train_label_lite.csv')
hasher = pd.read_csv('temp_data/test_id.csv')
label_map = {'retained':0,'2month':1,'month':2,'week':3}
y_train = pd.Series([label_map[l] for l in train_label.label])
inv_map = {label_map[k]:k for k in label_map.keys()}

In [54]:
pd.read_csv('X_test_final.csv').shape

(100000, 450)

---

### 5-fold cross validation XGB

In [6]:
#### xgb
grid_result = []
param = {}
#### XGB parameters
## General Parameters
param['n_gpus'] = -1
param['tree_method'] = 'gpu_hist'
param['silent'] = 0

## Booster Parameters
param['n_estimators'] = 2240 
param['learning_rate'] = 0.01
param['min_child_weight'] = 6.5
param['max_depth'] = 13
param['gamma'] = 0.1
param['reg_alpha'] =0.01
param['reg_lambda'] = 0.05
param['subsample'] = 0.9
param['colsample_bytree'] = 0.75
param['scale_pos_weight'] = 1

## Learning task parameters
param['num_class'] = 4
param['objective'] = 'multi:softmax'
param['seed'] = 7

## update?
#param['process_type'] = 'update'
#param['updater'] = 'refresh'
#param['refresh_leaf'] = True

model = xgb.XGBClassifier(**param)

In [7]:
#### cross validation(for NA)
kfold = StratifiedKFold(n_splits = 5 ,random_state = 7,shuffle=True).split(X_train, y_train)
scores = []
predict_set = []
for k, (train, test) in enumerate(kfold):
    model.fit(X_train.iloc[train,:], y_train[train],eval_metric = F1)  ####요부분 trimming
    score = model.score(X_train.iloc[test,:], y_train[test])
    scores.append(score)
    print('Fold: %s, Class dist.: %s, F1: %.3f' % (k+1,np.bincount(y_train[train]), score))
    ### predict
    y_pred = model.predict(X_train.iloc[test,:])
    predict_set += [(x,inv_map[y_pred[i]]) for i,x in enumerate(test)]
print('\nCV F1: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

Fold: 1, Class dist.: [20000 20000 20000 20000], F1: 0.743
Fold: 2, Class dist.: [20000 20000 20000 20000], F1: 0.740
Fold: 3, Class dist.: [20000 20000 20000 20000], F1: 0.747
Fold: 4, Class dist.: [20000 20000 20000 20000], F1: 0.746
Fold: 5, Class dist.: [20000 20000 20000 20000], F1: 0.739

CV F1: 0.743 +/- 0.003


---

### Fitting xgb

In [7]:
#### xgb
grid_result = []
param = {}
#### XGB parameters
## General Parameters
param['n_gpus'] = -1
param['tree_method'] = 'gpu_hist'
param['silent'] = 0

## Booster Parameters
param['n_estimators'] = 2240 #요기...
param['learning_rate'] = 0.01
param['min_child_weight'] = 6.5
param['max_depth'] = 13
param['gamma'] = 0.1
param['reg_alpha'] =0.01
param['reg_lambda'] = 0.05
param['subsample'] = 0.9
param['colsample_bytree'] = 0.75
param['scale_pos_weight'] = 1

## Learning task parameters
param['num_class'] = 4
param['objective'] = 'multi:softmax'
param['seed'] = 7

## update?
#param['process_type'] = 'update'
#param['updater'] = 'refresh'
#param['refresh_leaf'] = True

model = xgb.XGBClassifier(**param)

In [8]:
#### 전체 데이터 학습
model = xgb.XGBClassifier(**param)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.75, gamma=0.1, learning_rate=0.01,
       max_delta_step=0, max_depth=13, min_child_weight=6.5, missing=None,
       n_estimators=2240, n_gpus=-1, n_jobs=1, nthread=None, num_class=4,
       objective='multi:softprob', random_state=0, reg_alpha=0.01,
       reg_lambda=0.05, scale_pos_weight=1, seed=7, silent=0,
       subsample=0.9, tree_method='gpu_hist')

In [58]:
#### prob 클래스별 확률 예측
xgb_prob = pd.DataFrame(model.predict_proba(X_test),columns=['y_pred_prob0', 'y_pred_prob1', 'y_pred_prob2','y_pred_prob3'])
xgb_prob.to_csv('XGB_FINAL.csv',index = False)

ValueError: feature_names mismatch: ['differ_ratio_1', 'play_time_Count', 'cnt_enter_inzone_skilled_Count', 'making_cnt_Count', 'normal_chat_median_basic_time', 'first_week_cnt_clear_raid_light', 'cnt_enter_raid_light_range_stat', 'payment_amount_MA_5_basic_time', 'duel_cnt_sum_stat', 'trade_InDeg', 'partybattle_cnt_sum_stat', 'payment_amount_median_basic_time', 'partybattle_cnt_max_stat', 'payment_amount_kurt_basic_time', 'cnt_enter_raid_light_median_stat', 'guild_chat_MA_1_basic_time', 'whisper_chat_Count', 'cnt_enter_inzone_light_Count', 'payment_amount_MA_1_basic_time', 'cnt_enter_raid_light_max_stat', 'payment_amount_by_play_time', 'normal_chat_MA_2_basic_time', 'cnt_dt_Count', 'normal_chat_MA_1_basic_time', 'payment_amount_MA_3_basic_time', 'cnt_enter_raid_light_Count', 'district_chat_max_stat', 'payment_amount_min_stat', 'first_week_making_cnt', 'payment_amount_MA_4_basic_time', 'cnt_use_buffitem_Count', 'game_combat_time_Count', 'first_week_cnt_clear_inzone_solo', 'party_cnt_wi

In [12]:
### 부스터 빼와서 5가지 feature importance 빼오기...
xgb_booster= model.get_booster()
types = ['weight','gain','cover','total_gain','total_cover']
result = pd.DataFrame({'feature':[k for k in xgb_booster.get_score().keys()]})

In [13]:
for tp in types:
    temp = xgb_booster.get_score(importance_type=tp)
    result= pd.merge(result,pd.DataFrame([(x,temp[x]) for x in temp],columns=['feature',tp]))

In [15]:
result.to_csv('전체.csv',index = False)

---

### 5 fold-cross validation extratree

In [22]:
from sklearn.ensemble import ExtraTreesClassifier

In [23]:
clf_etc=ExtraTreesClassifier(verbose=1,criterion='gini',max_depth = 19, max_features = 444, min_samples_leaf = 1,n_estimators=300,random_state= 7, n_jobs=-1)

In [24]:
def f1(X_val, y_val, model,mapping):
    """
    Model evaluation function for multiclass classification problem
    1) F-1 score, Precision, Recall
    2) ROC curve, PR curve는 추후에 생각
    """
 
    #### predict the value
    y_pred = model.predict(X_val)

    
    #print('-'*50)
    #print('2. F1-score')
    
    # inverse pre/ rec
    pre = precision_score(y_true = y_val, y_pred = y_pred, average=None)
    rec = recall_score(y_true = y_val, y_pred = y_pred, average=None)

    # f1 measure
    f1_score = 8/(sum(1/pre) + sum(1/rec))
    
    # view - precision recall
    table = pd.DataFrame([])

    for i,k in enumerate(mapping.keys()):
        table[k] = [pre[i],rec[i]]
    table.index = ['precision','recall']
    # print(table)
    
    # view - f1
    #print('F1_score %.3f'%f1_score)
    #print('='*50)
    return f1_score

In [26]:
X_train = X_train.fillna(0.0)

In [28]:
#### cross validation
kfold = StratifiedKFold(n_splits = 5 ,random_state = 7).split(X_train, y_train)
scores = []

for k, (train, test) in enumerate(kfold):
    clf_etc.fit(X_train.iloc[train,], y_train.iloc[train,])
    score = f1(X_train.iloc[test,], y_train.iloc[test],clf_etc,label_map)
    scores.append(score)
    print('Fold: %s, Class dist.: %s, F1: %.3f' % (k+1,np.bincount(y_train[train]), score))
    
print('\nCV F1: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.1s finished


Fold: 1, Class dist.: [20000 20000 20000 20000], F1: 0.723


[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.2s finished


Fold: 2, Class dist.: [20000 20000 20000 20000], F1: 0.733


[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.1s finished


...

[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished
[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.2s finished


Fold: 4, Class dist.: [20000 20000 20000 20000], F1: 0.722


[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished


Fold: 5, Class dist.: [20000 20000 20000 20000], F1: 0.717

CV F1: 0.724 +/- 0.005


[Parallel(n_jobs=40)]: Done 120 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done 300 out of 300 | elapsed:    0.1s finished


---

### extratree

In [29]:
clf_etc=ExtraTreesClassifier(verbose=1,criterion='gini',max_depth = 19, max_features = 444, min_samples_leaf = 1,n_estimators=300,random_state= 7, n_jobs=-1)

In [31]:
clf_etc.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.9min finished


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=19, max_features=444, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
           oob_score=False, random_state=7, verbose=1, warm_start=False)

In [65]:
X_test = X_test.fillna(0.0)

In [66]:
#### prob 클래스별 확률 예측
extra_prob = pd.DataFrame(model.predict_proba(X_test),columns=['y_pred_prob0', 'y_pred_prob1', 'y_pred_prob2','y_pred_prob3'])

ValueError: feature_names mismatch: ['differ_ratio_1', 'play_time_Count', 'cnt_enter_inzone_skilled_Count', 'making_cnt_Count', 'normal_chat_median_basic_time', 'first_week_cnt_clear_raid_light', 'cnt_enter_raid_light_range_stat', 'payment_amount_MA_5_basic_time', 'duel_cnt_sum_stat', 'trade_InDeg', 'partybattle_cnt_sum_stat', 'payment_amount_median_basic_time', 'partybattle_cnt_max_stat', 'payment_amount_kurt_basic_time', 'cnt_enter_raid_light_median_stat', 'guild_chat_MA_1_basic_time', 'whisper_chat_Count', 'cnt_enter_inzone_light_Count', 'payment_amount_MA_1_basic_time', 'cnt_enter_raid_light_max_stat', 'payment_amount_by_play_time', 'normal_chat_MA_2_basic_time', 'cnt_dt_Count', 'normal_chat_MA_1_basic_time', 'payment_amount_MA_3_basic_time', 'cnt_enter_raid_light_Count', 'district_chat_max_stat', 'payment_amount_min_stat', 'first_week_making_cnt', 'payment_amount_MA_4_basic_time', 'cnt_use_buffitem_Count', 'game_combat_time_Count', 'first_week_cnt_clear_inzone_solo', 'party_cnt_wi

In [ ]:
extra_prob.to_csv('extra_FINAL.csv',index = False)

In [33]:
extra_importance = clf_etc.feature_importances_
result = pd.DataFrame([(x,extra_importance[i]) for i, x in enumerate(X_train.columns.tolist())],columns=['feature','gain'])
result.to_csv('extra전체.csv',index = False)

---

### ensemble

In [51]:
extra_prob.shape

(100000, 4)

In [42]:
### mean ensemble
ensemble_prob= (xgb_prob + extra_prob)/2

In [43]:
ensemble_prob = ensemble_prob.idxmax(axis=1)

In [45]:
my_pred = [int(x[-1]) for x in ensemble_prob]

In [49]:
len(my_pred )

100000

In [ ]:
#### result
results = [inv_map[x] for x in my_pred]
result = pd.DataFrame({'acc_id':['te' + str(x) for x in X_test.index], 'label': results})
result['acc_id'] = [hasher.iloc[i,0] for i in range(len(result.acc_id))]